In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
from keras.datasets import mnist

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128  

In [3]:
from mnist import MNIST
data = MNIST(data_dir="data/MNIST/")

In [4]:
# The number of pixels in each dimension of an image.
img_size = data.img_size

# The images are stored in one-dimensional arrays of this length.
img_size_flat = data.img_size_flat

# Tuple with height and width of images used to reshape arrays.
img_shape = data.img_shape

# Number of classes, one class for each of 10 digits.
num_classes = 6

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = data.num_channels
print(img_size)
print(img_shape)
print(img_size_flat)
print(num_classes)
print(num_channels)

28
(28, 28)
784
6
1


In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

X_test = X_test.astype('float32')
X_train=X_train.astype('float32')
print(y_train.shape)

X_train05=[]
y_train05=[]
X_train6=[]
y_train6=[]
for i in range (0,60000):
    if(y_train[i]<6):
        X_train05.append(X_train[i])
        y_train05.append(y_train[i])
    else :
        X_train6.append(X_train[i])
        y_train6.append(y_train[i])
X_train05=np.array(X_train05)
y_train05=np.array(y_train05)
X_train6=np.array(X_train6)
y_train6=np.array(y_train6)

print(X_train05.shape)
print(y_train05.shape)

X_test05=[]
y_test05=[]
X_test6=[]
y_test6=[]
for i in range (0,10000):
    if(y_test[i]<6):
        X_test05.append(X_test[i])
        y_test05.append(y_test[i])
    else :
        X_test6.append(X_test[i])
        y_test6.append(y_test[i])
X_test05=np.array(X_test05)
y_test05=np.array(y_test05)
X_test6=np.array(X_test6)
y_test6=np.array(y_test6)

print(X_test05.shape)
print(y_test05.shape)

(60000,)
(36017, 784)
(36017,)
(6031, 784)
(6031,)


In [6]:
shape1 = [filter_size1, filter_size1, num_channels, num_filters1]
shape2 = [filter_size2, filter_size2, num_filters1, num_filters2]

wconv1=tf.Variable(tf.truncated_normal(shape1, stddev=0.05), name='wconv1')
bconv1=tf.Variable(tf.constant(0.05, shape=[num_filters1]), name='bconv1')

wconv2=tf.Variable(tf.truncated_normal(shape2, stddev=0.05), name='wconv2')
bconv2=tf.Variable(tf.constant(0.05, shape=[num_filters2]), name='bconv2')

wfc1=tf.Variable(tf.truncated_normal(shape=[1764,128], stddev=0.05), name='wfc1')
bfc1=tf.Variable(tf.constant(0.05, shape=[128]), name='bfc1')

wfc2=tf.Variable(tf.truncated_normal(shape=[128,6], stddev=0.05), name='wfc2')
bfc2=tf.Variable(tf.constant(0.05, shape=[6]), name='bfc2')

In [7]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels], name='x_image')
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [8]:
#creating model 
layer1 = tf.nn.conv2d(input=x_image,
                         filter=wconv1,
                         strides=[1, 1, 1, 1],
                         padding='SAME', name='l1')
layer1+=bconv1
layer2 = tf.nn.max_pool(value=layer1,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME', name='l2')

layer3 = tf.nn.relu(layer2,name='l3')

layer4 = tf.nn.conv2d(input=layer3,
                         filter=wconv2,
                         strides=[1, 1, 1, 1],
                         padding='SAME', name='l4')
layer4+=bconv2
layer5 = tf.nn.max_pool(value=layer4,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME', name='l5')

layer6 = tf.nn.relu(layer5, name='l6')
layer7 = tf.reshape(layer6, [-1, 1764])
layer8 = tf.matmul(layer7, wfc1) + bfc1
layer9=tf.nn.relu(layer8, name='l9')
layer10 = tf.matmul(layer9, wfc2) + bfc2
layer11=tf.nn.relu(layer10, name='l11')

In [9]:
y_pred = tf.nn.softmax(layer11)
y_pred_cls = tf.argmax(y_pred, axis=1)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer11,
                                                        labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init_op = tf.global_variables_initializer()
train_batch_size = 64

In [10]:
from keras.utils import np_utils
y_train05 = np_utils.to_categorical(y_train05)
y_test05= np_utils.to_categorical(y_test05)
y_train6 = np_utils.to_categorical(y_train6)
y_test6= np_utils.to_categorical(y_test6)

In [12]:
k=0
sess=tf.Session()
    # initialise the variables
sess.run(init_op)
batch_size=64
total_batch = int(X_train05.shape[0] / batch_size)
for epoch in range(10):
    avg_cost = 0
    k=0
    for i in range(total_batch):
        batch_x = X_train05[k:k+batch_size,:]
        batch_y = y_train05[k:k+batch_size,:]
        k+=64
        _, c = sess.run([optimizer, cost], 
                            feed_dict={x: batch_x, y_true: batch_y})
        avg_cost += c / total_batch
    test_acc = sess.run(accuracy, 
                       feed_dict={x: X_test05, y_true: y_test05})
    print("Epoch:", (epoch + 1)," test accuracy: {:.3f}".format(test_acc))
    print(avg_cost)
print("\nTraining complete!")
saver = tf.train.Saver()
print(sess.run(accuracy, feed_dict={x: X_test05, y_true: y_test05}))
save_path = saver.save(sess, "/tmp/model.ckpt")
print("Model saved in path: %s" % save_path)
    

Epoch: 1  test accuracy: 0.831
0.8084652654004686
Epoch: 2  test accuracy: 0.843
0.6226834472918421
Epoch: 3  test accuracy: 0.845
0.596993181470026
Epoch: 4  test accuracy: 0.848
0.38054037014572684
Epoch: 5  test accuracy: 0.848
0.28812371223528627
Epoch: 6  test accuracy: 0.850
0.28147243912533937
Epoch: 7  test accuracy: 0.848
0.27766485000526314
Epoch: 8  test accuracy: 0.849
0.2758113247909156
Epoch: 9  test accuracy: 0.849
0.2741463653266005
Epoch: 10  test accuracy: 0.847
0.2757125372113495

Training complete!
0.84695715
Model saved in path: /tmp/model.ckpt


In [13]:
saver = tf.train.Saver()
saver.save(sess, 'test_model')

'test_model'

In [16]:
a=bconv1.eval(session=sess)
print(a)

[0.03537654 0.04211676 0.04512154 0.05142858 0.03888073 0.04180032
 0.04896171 0.03315908 0.04257441 0.05490109 0.04544467 0.04622808
 0.04538419 0.05079068 0.04108493 0.04909592]
